In [134]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.DataFrame(pd.read_csv('March iHeart Everyone HR Pull.xlsx - Sheet1.csv'))
empl_roll = pd.DataFrame(pd.read_csv('CR Indexed Worker Roster - Sales 2021-01-04 19_16 CST.xlsx - Sheet1.csv'))
user_df = pd.DataFrame(pd.read_csv('ihm_users.csv'))
market_owners = pd.DataFrame(pd.read_csv('ihm_account_management.csv'))
workday = pd.DataFrame(pd.read_csv('usersChanges_2021_02_15_11_54.csv'))
revenue = pd.DataFrame(pd.read_csv('report1619705426780.csv'))
projects = pd.DataFrame(pd.read_csv('iheart_projects.csv', error_bad_lines=False))
newhr = pd.DataFrame(pd.read_csv('March iHeart Everyone HR Pull.xlsx - Sheet1.csv'))
accounts = pd.DataFrame(pd.read_csv('report1618865683080.csv'))['Account Name']
ids = pd.DataFrame(pd.read_csv('report1621283786609.csv'))
print('The shape of revenue is: ' + str(revenue.shape))

The shape of revenue is: (19620, 16)


b'Skipping line 1989: expected 12 fields, saw 13\nSkipping line 5213: expected 12 fields, saw 17\nSkipping line 13571: expected 12 fields, saw 14\nSkipping line 14945: expected 12 fields, saw 13\nSkipping line 16603: expected 12 fields, saw 13\nSkipping line 17672: expected 12 fields, saw 14\nSkipping line 21734: expected 12 fields, saw 13\nSkipping line 22057: expected 12 fields, saw 13\n'


In [135]:
def Differences(hrnames, accountnames):
    li_diff = [i for i in hrnames + accountnames if i not in hrnames or i not in accountnames]
    missingfromhr = [i for i in hrnames + accountnames if i not in hrnames]
    missingfromaccountnames = [i for i in hrnames + accountnames if i not in accountnames]
    if len(missingfromaccountnames) != 0:
        return 'missing from Salesforce account names:', missingfromaccountnames
    else:
        return print('full agreement')

Altering field names of tables to match Salesforce fields

In [136]:
empl_roll.rename(columns = {'Worker': 'hr_worker', 'Job Title': 'hr_job_title', 'Business Title': 'hr_business_title',
                                       'Hire Date': 'hr_hire_date', 'Phone - Primary Work': 'hr_work_phone',
                                       'Mobile Phones': 'hr_mobile_phone', "Worker's Manager": 'hr_worker_manager',
                                        "Worker's Manager Business Title": "hr_worker_manager_business_title",
                                       "Worker's Manager's Manager": "hr_worker_manager_manager",
                                       "Worker's Manager's Manager Business Title": "hr_worker_manager_manager_business_title",
                                       'Market': 'hr_market', 'Market President': 'hr_market_president', 'Region/Area': 'hr_region_area',
                                       'Area/Region/Community President': 'hr_area_region_community_president', 'Division': 'hr_division',
                                       'Division President': 'hr_division_president'}, inplace=True)

user_df.rename(columns = {'client': 'adwire_client', 'group_x': 'adwire_group', 'name_x': 'adwire_name_x',
                                       'email': 'adwire_email', 'projects': 'adwire_projects', 'spend': 'adwire_spend',
                                       'project_ts': 'adwire_first_posting', 'group_cap': 'adwire_cap', 'owner': 'account_owner',
                                        'group_upper': 'adwire_group_upper'}, inplace=True)

workday.rename(columns = {'first name_x': 'workday_first_name', 
                                       'action': 'workday_action', 'client code': 'workday_client_code', 'first name_y': 'workday_first_name2',
                                       'last name': 'workday_last_name', 'group_y': 'workday_group_y'}, inplace=True)

revenue.rename(columns = {'Opportunity: Opportunity ID': 'iHeart Opportunity ID',
                            'Opportunity: Stage': 'Stage',
                           'Opportunity: Opportunity Owner: Market Division': 'Revenue Market Division',
                           'Opportunity: Opportunity Owner: Market Area': 'Revenue Market Area',
                           'Opportunity: Opportunity Owner: Home Operating Market': 'Account Name',
                           'Opportunity: Opportunity Owner: Full Name': 'iHeart Oppty Owner',
                           'OppOwnerEmail': 'email',
                           'New Business': 'new_business',
                           'Opportunity: Account Name: Account Name': 'opp_account_name',
                           'Opportunity: Opportunity Name': 'Opportunity Name',
                           'Adwire Project ID': 'project_id',
                           'Form Date': 'form_date',
                           'Opportunity: Created Date': 'Created Date',
                           'Opportunity: Close Date': 'Close Date',
                           'Opportunity: Gross Expected Revenue': 'Expected iHeart Revenue',
                           'Opportunity: Gross Amount': 'iHeart Gross Revenue',
                           'Form Created on Closed Opp': 'Created on Closed Opp',
                           'Opportunity: Opportunity Owner: Market Type': 'Revenue Market Type'}, inplace=True)

projects.rename(columns = {'project_id': 'project_id', 'adwire_post_date': 'project_post_date', 'email': 'creator_email', 'title': 'project_name',
                           'name': 'Product', 'name.1': 'project_creator', 'create_ts': 'project_post_date', 'name_y': 'projects_name_y', 'group': 'project_group',
                                        'submissions': 'projects_submissions', 'opportynity_id': 'project_opportunity_id',
                                       'credits_used': 'project_credits_used', 'industry': 'project_industry', 'tone': 'project_tone'}, inplace=True)

In [137]:
# Aligning market names to match Salesforce account names

#############
#############

newhr['User - Division'] = ['iHeart Media - ' + x.replace(' - Market', '') for x in newhr['User - Division']]
newhr.loc[(newhr['User - Division'] == 'iHeart Media - Inside Sales & Bus Development'), 'User - Division'] = 'iHeart Media - Inside Sales'

print('newhr: ' + str(Differences(newhr['User - Division'].unique().tolist(), accounts.tolist())))

#############
#############

revenue['form_date'] = pd.to_datetime(revenue['form_date'], errors='coerce').dt.strftime('%m/%d/%Y')

revenue = revenue[(revenue.form_date > '02/19/2020')]
# revenue = revenue[(revenue.form_date < '04/28/2021')]

re_outer = re.compile(r'([^A-Z ])([A-Z])')
re_inner = re.compile(r'\b[A-Z]+(?=[A-Z][a-z])')
revenue['Account Name'] = ['iHeart Media - ' + re_inner.sub(r'\g<0> ', re_outer.sub(r'\1 \2', x)) for x in revenue['Account Name'].astype(str)]
revenue['Account Name'] = [x[:-3] + ', ' + x[-2:] for x in revenue['Account Name']]

revenue.loc[(revenue['Account Name'] == 'iHeart Media - St Petersburg, FL'),'Account Name'] = 'iHeart Media - Inside Sales'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Fort Myers, FL'), 'Account Name'] = 'iHeart Media - Fort Myers/Naples, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Cedar Rapids, IA'), 'Account Name'] = 'iHeart Media - Cedar Rapids/Iowa City, IA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Panama City, FL'), 'Account Name'] = 'iHeart Media - Panama City/Tallahassee, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Roanoke, VA'), 'Account Name'] = 'iHeart Media - Roanoke/Lynchburg, VA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Sarasota, FL'), 'Account Name'] = 'iHeart Media - Sarasota/Bradenton, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Ashland, OH'),'Account Name'] = 'iHeart Media - Ashland/Mansfield, OH'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - New York, NY'),'Account Name'] = 'iHeart Media - New York City, NY'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Modesto, CA'),'Account Name'] = 'iHeart Media - Modesto/Stockton, CA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Hattiesburg, MS'),'Account Name'] = 'iHeart Media - Laurel, MS'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Albany, GA'),'Account Name'] = 'iHeart Media - Columbus, GA'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Mc Allen, TX'),'Account Name'] = 'iHeart Media - McAllen, TX'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - , an'),'Account Name'] = 'iHeart Media - No Data'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Ta, pa'),'Account Name'] = 'iHeart Media - Tampa, FL'
revenue.loc[(revenue['Account Name'] == 'iHeart Media - Bay Area, CA'),'Account Name'] = 'iHeart Media - San Francisco, CA'

revenue = pd.merge(revenue, ids[['Email', 'Account ID']], left_on='email', right_on='Email', how='left')

print('revenue: ' + str(Differences(revenue['Account Name'].unique().tolist(), accounts.tolist())))

#############
#############

user_df.group = ['iHeart Media - ' + x for x in user_df.group]
user_df.group = user_df.group.str.strip()

user_df.adwire_first_posting = pd.to_datetime(user_df.adwire_first_posting, errors='coerce').dt.strftime('%m/%d/%Y')

user_df.loc[(user_df.group == 'iHeart Media - St. Petersburg, FL | Inside Sales'), 'group'] = 'iHeart Media - Inside Sales'
user_df.loc[(user_df.group == 'iHeart Media - Lexington / Somerset, KY'), 'group'] = 'iHeart Media - Lexington/Somerset, KY'
user_df.loc[(user_df.group == 'iHeart Media - Roanoke / Lynchburg, VA'), 'group'] = 'iHeart Media - Roanoke/Lynchburg, VA'
user_df.loc[(user_df.group == 'iHeart Media - Modesto / Stockton, CA'), 'group'] = 'iHeart Media - Modesto/Stockton, CA'

print('user_df: ' + str(Differences(user_df.group.unique().tolist(), accounts.tolist())))

#############
#############

projects.project_post_date = pd.to_datetime(projects.project_post_date)
projects['project_post_date'] = pd.to_datetime(projects['project_post_date'], errors='coerce').dt.strftime('%m/%d/%Y')
projects = projects[(projects.project_post_date > '02/19/2020')]
# projects = projects[(projects.project_post_date < '04/28/2021')]

projects['project_opportunity_id'] = projects['project_opportunity_id'].map(lambda x: str(x)[:-3])

projects = projects[projects.project_opportunity_id.map(len) == 15]
projects.creator_email = projects.creator_email.str.lower()

projects.project_group = ['iHeart Media - ' + x for x in projects.project_group]
projects.project_group = projects.project_group.str.strip()

projects = projects.loc[(projects.project_group != 'iHeart Media - 9') & (projects.project_group != 'iHeart Media - 6')]

projects.loc[(projects.project_group == 'iHeart Media - St. Petersburg, FL | Inside Sales'), 'project_group'] = 'iHeart Media - Inside Sales'
projects.loc[(projects.project_group == 'iHeart Media - Lexington / Somerset, KY'), 'project_group'] = 'iHeart Media - Lexington/Somerset, KY'
projects.loc[(projects.project_group == 'iHeart Media - Roanoke / Lynchburg, VA'), 'project_group'] = 'iHeart Media - Roanoke/Lynchburg, VA'
projects.loc[(projects.project_group == 'iHeart Media - Modesto / Stockton, CA'), 'project_group'] = 'iHeart Media - Modesto/Stockton, CA'

projects = pd.merge(projects, user_df[['adwire_email','adwire_first_posting', 'adwire_cap']], left_on = 'creator_email', right_on = 'adwire_email', how='left')
projects = pd.merge(projects, ids[['Email', 'Contact ID', 'Account ID']], left_on = 'creator_email', right_on = 'Email', how='left')

print('projects: ' + str(Differences(projects.project_group.unique().tolist(), accounts.tolist())))
print('projects lower limit: ' + projects.project_post_date.min(), 'projects upper limit: ' + projects.project_post_date.max())
print('revenue lower limit: ' + revenue.form_date.min(), 'revenue upper limit: ' + revenue.form_date.max())
print('revenue shape is: ' + str(revenue.shape))
print('the shape of projects is : ' + str(projects.shape))

full agreement
newhr: None
revenue: ('missing from Salesforce account names:', ['iHeart Media - No Data'])
user_df: ('missing from Salesforce account names:', ['iHeart Media - Unsorted Users'])
full agreement
projects: None
projects lower limit: 02/19/2021 projects upper limit: 12/31/2020
revenue lower limit: 02/19/2021 revenue upper limit: NaT
revenue shape is: (17168, 18)
the shape of projects is : (17663, 18)


In [138]:
# Essentially a VLOOKUP; backfilling Account Name and Revenue Market Area fields with information from newhr because many of these values are consistently absent from
# the revenue report alone.

merge_newhr = newhr.rename(columns = {'User - User Email': 'email'})
merge_newhr['email'] = merge_newhr.email.str.lower()

revenue = revenue.merge(merge_newhr[['email', 'User - Division', 'User - Division Parent']], on='email', how='left')
revenue['Account Name'] = np.where(revenue['Account Name'] == 'iHeart Media - No Data', revenue['User - Division'], revenue['Account Name'])
revenue['Revenue Market Area'] = np.where(revenue['Revenue Market Area'].isnull(), revenue['User - Division Parent'], revenue['Revenue Market Area'])

revenue.drop('User - Division', axis=1, inplace=True)
revenue.drop('User - Division Parent', axis=1, inplace=True)

print('revenue: ' + str(Differences(revenue['Account Name'].unique().tolist(), accounts.tolist())))
print(revenue.shape)

revenue: ('missing from Salesforce account names:', [nan])
(17168, 18)


Mutating 3 columns:
- Writelabel Owner - Opportunity Owner (revenue)
- Writelabel Owner - Project Owner (projects)
- Writelabel Owner - User (user_df)
- Writelabel Owner (wl_owner) - Employee (newhr)

In [110]:
arliss = market_owners[market_owners['Owner'] == 'Arliss Coates']
arliss = ['iheart media - ' + x.lower() for x in arliss.Name.tolist()]
ashley = market_owners[market_owners['Owner'] == 'Ashley McCarthy']
ashley = ['iheart media - ' + x.lower() for x in ashley['Name'].tolist()]
jenny = market_owners[market_owners['Owner'] == 'Jenny Donheiser']
jenny = ['iheart media - ' + x.lower() for x in jenny['Name'].tolist()]
mariana = market_owners[market_owners['Owner'] == 'Mariana E Trevino Stafford']
mariana = ['iheart media - ' + x.lower() for x in mariana['Name'].tolist()]

newhr_conditions = [(newhr['User - Division'].str.lower().isin(arliss)),
              (newhr['User - Division'].str.lower().isin(ashley)),
              (newhr['User - Division'].str.lower().isin(mariana)),
              (newhr['User - Division'].str.lower().isin(jenny))]

revenue_conditions = [(revenue['Account Name'].str.lower().isin(arliss)),
              (revenue['Account Name'].str.lower().isin(ashley)),
              (revenue['Account Name'].str.lower().isin(mariana)),
              (revenue['Account Name'].str.lower().isin(jenny))]

projects_conditions = [(projects.project_group.str.lower().isin(arliss)),
              (projects.project_group.str.lower().isin(ashley)),
              (projects.project_group.str.lower().isin(mariana)),
              (projects.project_group.str.lower().isin(jenny))]

user_df_conditions = [(user_df.group.str.lower().isin(arliss)),
              (user_df.group.str.lower().isin(ashley)),
              (user_df.group.str.lower().isin(mariana)),
              (user_df.group.str.lower().isin(jenny))]

values = ['Arliss Coates', 'Ashley McCarthy', 'Mariana Trevino', 'Jenny Donheiser']

newhr['wl_owner'] = np.select(newhr_conditions, values)
revenue['wl_owner'] = np.select(revenue_conditions, values)
projects['wl_owner'] = np.select(projects_conditions, values)
user_df['wl_owner'] = np.select(user_df_conditions, values)

In [111]:
# revenue.loc[(revenue['Form Date'] > '20210402') & (revenue['Form Date'] != 'NaT')]
# projects[projects.apply(lambda row: row.astype(str).str.contains('converse', case=False).any(), axis=1)]
newhr[newhr.apply(lambda row: row.astype(str).str.contains('Koch', case=False).any(), axis=1)]
# empl_roll[empl_roll.apply(lambda row: row.astype(str).str.contains('shosh', case=False).any(), axis = 1)].hr_market.unique().tolist()
# revenue.shape

,First Name,Last Name,Name,User - Division Parent,User - Division,User - Position,User - User Email,User - Manager - User Full Name,User - Manager - User Email,wl_owner
273,Joshua,Koch,Joshua Koch,Ohio/West Virginia - Area,"iHeart Media - Chillicothe, OH",Market President,JoshuaKoch@iheartmedia.com,"Bell, Matt",MattBell@iheartmedia.com,Arliss Coates
274,John,Barney,John Barney,Ohio/West Virginia - Area,"iHeart Media - Chillicothe, OH",Outside Account Executive - 100% Commission,JohnBarney@iheartmedia.com,"Koch, Joshua",JoshuaKoch@iheartmedia.com,Arliss Coates
275,Laura,Strausbaugh,Laura Strausbaugh,Ohio/West Virginia - Area,"iHeart Media - Chillicothe, OH",Outside Account Executive - 100% Commission,LauraStrausbaugh@iheartmedia.com,"Koch, Joshua",JoshuaKoch@iheartmedia.com,Arliss Coates


In [112]:
projects.to_csv('projects_for_salesforce.csv')
revenue.to_csv('revenue_for_salesforce.csv')

In [113]:
print(revenue.shape)
print(projects.shape)

(17168, 17)
(4500, 13)


In [114]:
user_df.head()
# projects.head()

,adwire_client,group,name,adwire_email,adwire_projects,adwire_spend,adwire_first_posting,adwire_cap,wl_owner
0,iHeart Media,"iHeart Media - Tampa, FL",Kelly Thorson,kellythorson@iheartmedia.com,41,43.5,10/23/2018,35,Mariana Trevino
1,iHeart Media,"iHeart Media - Tampa, FL",Lisa Converse,lisaconverse@iheartmedia.com,30,32.0,12/11/2018,35,Mariana Trevino
2,iHeart Media,"iHeart Media - Tampa, FL",Mary Weber,maryweber@iheartmedia.com,10,15.5,10/29/2018,35,Mariana Trevino
3,iHeart Media,"iHeart Media - Tampa, FL",Michelle Land,michelleland@iheartmedia.com,21,24.0,10/24/2018,35,Mariana Trevino
4,iHeart Media,"iHeart Media - Tampa, FL",Shannon Gibson,shannongibson@iheartmedia.com,37,39.0,10/26/2018,35,Mariana Trevino


In [120]:
test.head()
ids.head()

,Salutation,Contact ID,First Name,Last Name,Email,Title,Account Name,Mailing City,Account Owner,Account ID
0,NaN,003f400001JDUZM,Tara,Adamos,taraadamos@iheartmedia.com,"SVP, Client Solutions",iHeart Media,New York,James Donahue,001f400000Wfz6z
1,NaN,003f4000019DmRd,Leslie,Greene-Faust,lesliegreene-faust@iheartmedia.com,Account Executive,iHeart Media,New York,James Donahue,001f400000Wfz6z
2,NaN,003f4000019DmRe,Stacey,Dobak,staceydobak@iheartmedia.com,Account Executive,iHeart Media,New York,James Donahue,001f400000Wfz6z
3,NaN,003f4000019DmRg,Madelyne,Schermer,madelyneandkim@iheartmedia.com,Account Executive,iHeart Media,New York,James Donahue,001f400000Wfz6z
4,NaN,003f4000019DmRh,Nicole,Buschey,nicholebuschey@iheartmedia.com,Account Executive,iHeart Media,New York,James Donahue,001f400000Wfz6z


In [10]:
start_date = "02/19/2020"
end_date = "04/28/2021"

after_start_date = revenue.form_date >= start_date
before_end_date = revenue.form_date <= end_date
revenue_between = after_start_date & before_end_date
filtered_dates = revenue.loc[revenue_between]

In [11]:
filtered_dates.shape

(4403, 17)

In [12]:
results1 = pd.DataFrame(pd.read_csv('Massive Upsert Results 1.csv'))
results1_orig = pd.DataFrame(pd.read_csv('Massive Upsert Results 1.csv'))
results2 = pd.DataFrame(pd.read_csv('Massive Upsert Results 2.csv'))

In [13]:
results1.head()

,Id,Success,Created,Error
0,a0F5G00000MsSN3UAN,True,False,NaN
1,a0F5G00000MsSNIUA3,True,True,NaN
2,a0F5G00000MsSNJUA3,True,True,NaN
3,a0F5G00000MsSNKUA3,True,True,NaN
4,a0F5G00000MsSNLUA3,True,True,NaN


In [91]:
results2.head()

,Id,Success,Created,Error
0,a0F5G00000MsSYPUA3,True,True,NaN
1,a0F5G00000MsSYQUA3,True,True,NaN
2,a0F5G00000MsSYRUA3,True,True,NaN
3,a0F5G00000MsSYSUA3,True,True,NaN
4,a0F5G00000MsSYTUA3,True,True,NaN


In [92]:
results1.Error = [str(x).replace('INVALID_FIELD:Foreign key external ID: ', '') for x in results1.Error]
results1.Error = [str(x).replace(' not found for field iHeart_Opportunity_ID__c in entity Opportunity:--', '') for x in results1.Error]


In [93]:
restults1 = results1.loc[~results1.Error.str.contains('006')]
opp_ids1 = results1.loc[results1.Error.str.contains('006')]
opp_ids1.to_csv('results_error_opportunities_1.csv')

In [94]:
restults1.Error =[str(x)[:50] for x in restults1.Error]
restults1.Error.unique()

array(['nan', '6123456789012',
       'FIELD_INTEGRITY_EXCEPTION:Specified External ID fo',
       'DUPLICATE_VALUE:Duplicate external id specified: B',
       'DUPLICATE_VALUE:Duplicate external id specified: E',
       'DUPLICATE_VALUE:Duplicate external id specified: V',
       'DUPLICATE_VALUE:Duplicate external id specified: S',
       'DUPLICATE_VALUE:Duplicate external id specified: C',
       'DUPLICATE_VALUE:Duplicate external id specified: A',
       'DUPLICATE_VALUE:Duplicate external id specified: G',
       'DUPLICATE_VALUE:Duplicate external id specified: D',
       'DUPLICATE_VALUE:Duplicate external id specified: T',
       'DUPLICATE_VALUE:Duplicate external id specified: M',
       'DUPLICATE_VALUE:Duplicate external id specified: H',
       'DUPLICATE_VALUE:Duplicate external id specified: L',
       'DUPLICATE_VALUE:Duplicate external id specified: 6',
       'DUPLICATE_VALUE:Duplicate external id specified: F',
       'DUPLICATE_VALUE:Duplicate external id specifie

In [ ]:
results

In [303]:
results2.Error = [str(x).replace('INVALID_FIELD:Foreign key external ID: ', '') for x in results2.Error]
results2.Error = [str(x).replace(' not found for field iHeart_Opportunity_ID__c in entity Opportunity:--', '') for x in results2.Error]

In [294]:
matched = pd.DataFrame(pd.read_csv('report1619705426780 - Sortable.csv'))
matched.dropna(inplace=True)
matched.rename(columns={"AdWire Project ID's": 'project_id'}, inplace=True)
matched_revenue = matched.merge(revenue, on='project_id', how='left')

In [297]:
matched_revenue.to_csv('matched_revenue.csv')

In [ ]:
results2.Error.unique().tolist()

In [32]:
projects.head()

,project_id,project_post_date,creator_email,project_name,Product,project_creator,project_group,projects_submissions,project_opportunity_id,project_credits_used,project_industry,project_tone,first_posting,wl_owner
3785,AQphaAagAAA,02/20/2020,nikkihilton@iheartmedia.com,InteCore PT,AdWire Draft Scripts | 60 Seconds,Nikki Hilton,"iHeart Media - Harrisburg, PA",8,0060y00001Az6v8,1.5,Medical and Healthcare,Funny,02/19/2021,Arliss Coates
3786,AQpqaAMwAAA,02/20/2020,nikkihilton@iheartmedia.com,Jim Coleman Honda,AdWire Draft Scripts | 60 Seconds,Nikki Hilton,"iHeart Media - Harrisburg, PA",9,0060y00001BgrKB,1.5,Automotive,Funny,02/19/2021,Arliss Coates
3787,AQs0VAMwAAA,02/20/2020,laurarhodes@iheartmedia.com,The Exterior Company,AdWire Draft Scripts | 30 Seconds,Laura Rhodes Grogan,"iHeart Media - Harrisburg, PA",12,0060y00001D2LN7,1,Building and Hardware,I don't care as long as it's good,02/19/2021,Arliss Coates
3788,AQs8FAMwAAA,02/20/2020,laurarhodes@iheartmedia.com,Korealicious,AdWire Draft Scripts | 15 Seconds,Laura Rhodes Grogan,"iHeart Media - Harrisburg, PA",13,0060y00001B0cIq,1,Restaurant / Food and Beverage,I don't care as long as it's good,02/19/2021,Arliss Coates
3789,AQtmXAMwAAA,02/20/2020,amyhathaway@iheartmedia.com,"Today's Home and Leisure Products, Inc.",AdWire Draft Scripts | 15 Seconds,Amy Hathaway,"iHeart Media - Harrisburg, PA",9,0060y00001BhUZ1,1,Travel and Lifestyle,I don't care as long as it's good,02/19/2021,Arliss Coates


In [127]:
ids.head()

,Salutation,Contact ID,First Name,Last Name,Email,Title,Account Name,Mailing City,Account Owner,Account ID
0,NaN,003f400001JDUZM,Tara,Adamos,taraadamos@iheartmedia.com,"SVP, Client Solutions",iHeart Media,New York,James Donahue,001f400000Wfz6z
1,NaN,003f4000019DmRd,Leslie,Greene-Faust,lesliegreene-faust@iheartmedia.com,Account Executive,iHeart Media,New York,James Donahue,001f400000Wfz6z
2,NaN,003f4000019DmRe,Stacey,Dobak,staceydobak@iheartmedia.com,Account Executive,iHeart Media,New York,James Donahue,001f400000Wfz6z
3,NaN,003f4000019DmRg,Madelyne,Schermer,madelyneandkim@iheartmedia.com,Account Executive,iHeart Media,New York,James Donahue,001f400000Wfz6z
4,NaN,003f4000019DmRh,Nicole,Buschey,nicholebuschey@iheartmedia.com,Account Executive,iHeart Media,New York,James Donahue,001f400000Wfz6z


In [128]:
revenue.head()

,iHeart Opportunity ID,Stage,Revenue Market Division,Revenue Market Area,Account Name,iHeart Oppty Owner,email,Opportunity Name,project_id,form_date,Created Date,Close Date,Expected iHeart Revenue,iHeart Gross Revenue,Created on Closed Opp,Revenue Market Type
71,0060y00001Az6v8,Closed - Won,Lankford,Pennsylvania Area,"iHeart Media - Harrisburg, PA",Nikki Hilton,nikkihilton@iheartmedia.com,Intecore P-Intecore PT-Digital Jan-Aug 2020-Hi...,AQphaAagAAA,02/20/2020,8/6/2019,8/6/2019,60000.0,60000.0,1,Community
72,0060y00001D2LN7,Closed - Won,Lankford,Pennsylvania Area,"iHeart Media - Harrisburg, PA",Laura Rhodes,laurarhodes@iheartmedia.com,The Exteri-Allentown Craig Stevens B104 March-...,AQs0VAMwAAA,02/20/2020,2/13/2020,3/10/2020,26835.0,26835.0,0,Community
73,0060y00001B0cIq,Closed - Won,Lankford,Pennsylvania Area,"iHeart Media - Harrisburg, PA",Laura Rhodes,laurarhodes@iheartmedia.com,"Korealicio-Ken Matthews-RHODES, LAURA-HarrPA-7...",AQs8FAMwAAA,02/20/2020,8/26/2019,11/22/2019,19770.0,19770.0,1,Community
74,0060y00001BhUZ1,Closed - Won,Lankford,Pennsylvania Area,"iHeart Media - Harrisburg, PA",Amy Hathaway,amyhathaway@iheartmedia.com,"Todays Hom-Q1-Q2 2020 Auction Trade-HATHAWAY, ...",AQtmXAMwAAA,02/20/2020,10/22/2019,2/24/2020,42995.0,42995.0,0,Community
75,0060y00001CDutg,Closed - Won,Lankford,Pennsylvania Area,"iHeart Media - Harrisburg, PA",Sarah Sweitzer,sarahsweitzer@iheartmedia.com,Wall Harri-WRBT/HP/HBG Wall and Floor 2020 Tra...,AQt0AAagAAA,02/20/2020,11/20/2019,2/20/2020,9000.0,9000.0,1,Community
